# Importing Required Libaries:

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow
import keras

import re
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM,Bidirectional,Embedding,Dense,Activation,SpatialDropout1D
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Read the Data:

In [3]:
data=pd.read_csv(r"/content/drive/MyDrive/DeepLearningProjects/Sentimental Analysis for Hotel Reviews/Datasets/Hotel_Reviews.csv")

# Data Description:

In [4]:
data.head(5)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [5]:
data.tail(5)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
515733,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/30/2015,8.1,Atlantis Hotel Vienna,Kuwait,no trolly or staff to help you take the lugga...,14,2823,location,2,8,7.0,"[' Leisure trip ', ' Family with older childre...",704 day,48.203745,16.335677
515734,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/22/2015,8.1,Atlantis Hotel Vienna,Estonia,The hotel looks like 3 but surely not 4,11,2823,Breakfast was ok and we got earlier check in,11,12,5.8,"[' Leisure trip ', ' Family with young childre...",712 day,48.203745,16.335677
515735,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/19/2015,8.1,Atlantis Hotel Vienna,Egypt,The ac was useless It was a hot week in vienn...,19,2823,No Positive,0,3,2.5,"[' Leisure trip ', ' Family with older childre...",715 day,48.203745,16.335677
515736,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/17/2015,8.1,Atlantis Hotel Vienna,Mexico,No Negative,0,2823,The rooms are enormous and really comfortable...,25,3,8.8,"[' Leisure trip ', ' Group ', ' Standard Tripl...",717 day,48.203745,16.335677
515737,Wurzbachgasse 21 15 Rudolfsheim F nfhaus 1150 ...,168,8/9/2015,8.1,Atlantis Hotel Vienna,Hungary,I was in 3rd floor It didn t work Free Wife,13,2823,staff was very kind,6,1,8.3,"[' Leisure trip ', ' Family with young childre...",725 day,48.203745,16.335677


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 515738 entries, 0 to 515737
Data columns (total 17 columns):
 #   Column                                      Non-Null Count   Dtype  
---  ------                                      --------------   -----  
 0   Hotel_Address                               515738 non-null  object 
 1   Additional_Number_of_Scoring                515738 non-null  int64  
 2   Review_Date                                 515738 non-null  object 
 3   Average_Score                               515738 non-null  float64
 4   Hotel_Name                                  515738 non-null  object 
 5   Reviewer_Nationality                        515738 non-null  object 
 6   Negative_Review                             515738 non-null  object 
 7   Review_Total_Negative_Word_Counts           515738 non-null  int64  
 8   Total_Number_of_Reviews                     515738 non-null  int64  
 9   Positive_Review                             515738 non-null  object 
 

In [7]:
data.shape
print("No.of rows in the dataset:",data.shape[0])
print("No.of columns in the dataset:",data.shape[1])

No.of rows in the dataset: 515738
No.of columns in the dataset: 17


In [8]:
data.duplicated().sum()

526

In [9]:
dataset=data.drop_duplicates()

In [10]:
dataset.shape
print("No.of rows in the dataset:",dataset.shape[0])
print("No.of columns in the dataset:",dataset.shape[1])

No.of rows in the dataset: 515212
No.of columns in the dataset: 17


# Data Preprocessing:

In [11]:
dataset.head(5)

,Hotel_Address,Additional_Number_of_Scoring,Review_Date,Average_Score,Hotel_Name,Reviewer_Nationality,Negative_Review,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Positive_Review,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Tags,days_since_review,lat,lng
0,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Russia,I am so angry that i made this post available...,397,1403,Only the park outside of the hotel was beauti...,11,7,2.9,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
1,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,8/3/2017,7.7,Hotel Arena,Ireland,No Negative,0,1403,No real complaints the hotel was great great ...,105,7,7.5,"[' Leisure trip ', ' Couple ', ' Duplex Double...",0 days,52.360576,4.915968
2,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,Australia,Rooms are nice but for elderly a bit difficul...,42,1403,Location was good and staff were ok It is cut...,21,9,7.1,"[' Leisure trip ', ' Family with young childre...",3 days,52.360576,4.915968
3,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/31/2017,7.7,Hotel Arena,United Kingdom,My room was dirty and I was afraid to walk ba...,210,1403,Great location in nice surroundings the bar a...,26,1,3.8,"[' Leisure trip ', ' Solo traveler ', ' Duplex...",3 days,52.360576,4.915968
4,s Gravesandestraat 55 Oost 1092 AA Amsterdam ...,194,7/24/2017,7.7,Hotel Arena,New Zealand,You When I booked with your company on line y...,140,1403,Amazing location and building Romantic setting,8,3,6.7,"[' Leisure trip ', ' Couple ', ' Suite ', ' St...",10 days,52.360576,4.915968


In [12]:
dataset.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng'],
      dtype='object')

In [13]:
Final_data=dataset.drop(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng'],axis=1)

In [14]:
Final_data.head(2)

,Negative_Review,Positive_Review
0,I am so angry that i made this post available...,Only the park outside of the hotel was beauti...
1,No Negative,No real complaints the hotel was great great ...


In [15]:
Final_data["Text"]=Final_data["Negative_Review"]+Final_data["Positive_Review"]
Final_data=Final_data.drop(["Negative_Review","Positive_Review"],axis=1)
Final_data.head(2)

,Text
0,I am so angry that i made this post available...
1,No Negative No real complaints the hotel was g...


In [16]:
Final_data["reviews"]=dataset["Reviewer_Score"].apply(lambda x:1 if x<5.0 else 0)
Final_data.head(2)

,Text,reviews
0,I am so angry that i made this post available...,1
1,No Negative No real complaints the hotel was g...,0


In [17]:
Final_data["reviews"].value_counts()

0    492980
1     22232
Name: reviews, dtype: int64

In [18]:
# Sampling the data:
bad_reviews = Final_data[Final_data["reviews"] == 1]
good_reviews = Final_data[Final_data["reviews"] == 0]

In [19]:
Final_data["reviews"].value_counts()

0    492980
1     22232
Name: reviews, dtype: int64

In [20]:
Final_data["Text"][1]

'No Negative No real complaints the hotel was great great location surroundings rooms amenities and service Two recommendations however firstly the staff upon check in are very confusing regarding deposit payments and the staff offer you upon checkout to refund your original payment and you can make a new one Bit confusing Secondly the on site restaurant is a bit lacking very well thought out and excellent quality food for anyone of a vegetarian or vegan background but even a wrap or toasted sandwich option would be great Aside from those minor minor things fantastic spot and will be back when i return to Amsterdam '

In [21]:
bad_review_data=bad_reviews.sample(22232)
good_review_data=good_reviews.sample(22232)

In [22]:
Final_data=pd.concat([bad_review_data,good_review_data],axis=0)

In [23]:
Final_data["reviews"].value_counts()

1    22232
0    22232
Name: reviews, dtype: int64

In [24]:
def case_conversion():
  Final_data.Text=[i.lower() for i in Final_data.Text.values]

def remove_specials():
  Final_data.Text=[re.sub(r"[^a-zA-Z]"," ",text) for text in Final_data.Text.values]

def remove_stopwords():
  stopwords_set=set(stopwords.words("english"))
  Final_data.Text=[word for word in Final_data.Text if word not in stopwords_set]

def find_rootword():
  lemmatizer=WordNetLemmatizer()
  Final_data.Text=[lemmatizer.lemmatize(word) for word in Final_data.Text.values]

def normalize_spaces():
  Final_data.Text=[re.sub(r"\s+"," ",text) for text in Final_data.Text.values]

In [25]:
case_conversion()
remove_specials()
remove_stopwords()
find_rootword()
normalize_spaces()

In [26]:
Final_dataset=Final_data["Text"]
Final_dataset

229653     i was provided with room with view on the str...
66234                                 everythingno positive
64392      when i arrived with my husband i was asked if...
185499     actually everything was more than perfect unt...
166192     hotel was worn out needs lots of fixing not w...
                                ...                        
334925     breakfast is like in hotel i love hotel room ...
102847     wifi patchy this hotel oozes character is ade...
32219      lift is not great located right by the tube s...
509803     bathroom could be a bit bigger rooms are at t...
243151     i thought the breakfast buffet was a bit disa...
Name: Text, Length: 44464, dtype: object

In [27]:
Final_dataset.iloc[1]

' everythingno positive'

# Data Transformation:

In [28]:
max_fatures = 2000
tokenizer = Tokenizer(num_words=max_fatures, split=' ')
tokenizer.fit_on_texts(Final_data['Text'].values)
X = tokenizer.texts_to_sequences(Final_data['Text'].values)
X = pad_sequences(X)
X

array([[   0,    0,    0, ...,   42,  243,   50],
       [   0,    0,    0, ...,    0,    0,   41],
       [   0,    0,    0, ...,   63,  867,    9],
       ...,
       [   0,    0,    0, ...,  165,    8, 1128],
       [   0,    0,    0, ...,    1,   14, 1395],
       [   0,    0,    0, ...,   18,   11,  397]], dtype=int32)

In [29]:
X.shape

(44464, 609)

# Model Training:

In [30]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 609, 128)          256000    
                                                                 
 spatial_dropout1d (SpatialD  (None, 609, 128)         0         
 ropout1D)                                                       
                                                                 
 lstm (LSTM)                 (None, 196)               254800    
                                                                 
 dense (Dense)               (None, 1)                 197       
                                                                 
Total params: 510,997
Trainable params: 510,997
Non-trainable params: 0
_________________________________________________________________
None


# creating training & testing data for Training:

In [31]:
y=Final_data["reviews"]
y

229653    1
66234     1
64392     1
185499    1
166192    1
         ..
334925    0
102847    0
32219     0
509803    0
243151    0
Name: reviews, Length: 44464, dtype: int64

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X,y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(29790, 609) (29790,)
(14674, 609) (14674,)


In [33]:
#history=model.fit(X_train,Y_train,validation_data=(X_test,Y_test),epochs=2,batch_size=32)

In [34]:
model.save(r"/content/drive/MyDrive/DeepLearningProjects/Sentimental Analysis for Hotel Reviews/Datasets/reviews.h5")

In [35]:
loaded_model=load_model(r"/content/drive/MyDrive/DeepLearningProjects/Sentimental Analysis for Hotel Reviews/Datasets/reviews.h5")

In [36]:
plt.plot(history.history["loss"],label="trainloss")
plt.plot(history.history["val_loss"],label="val_loss")
plt.legend()
plt.show()

NameError: ignored

In [ ]:
plt.plot(history.history["accuracy"],label="trainaccuracy")
plt.plot(history.history["val_accuracy"],label="val_accuracy")
plt.legend()
plt.show()

In [37]:
y_predict=loaded_model.predict(X_test)

459/459 [==============================] - 79s 164ms/step


In [38]:
y_predict

array([[0.49981993],
       [0.4990497 ],
       [0.49874786],
       ...,
       [0.4938686 ],
       [0.50000036],
       [0.49856776]], dtype=float32)

In [39]:
cm=confusion_matrix(Y_test,y_predict.round())
cm

array([[4773, 2579],
       [4199, 3123]])

In [43]:
reviews=[Final_data["Text"].iloc[0]]

In [44]:
reviews

[' i was provided with room with view on the street very noisy room you can hear everything from the street i asked receptionist to change my room he did it but higher floor also noisy i asked him again to change the room he refused because no rooms anymore at the same time booking com offered plenty options in the hotel i don t believe that no room it s just personal attitude of stuff i told him if he didn t change my room i will make bad rating for hotel and he told me i can go on i did it with pleasure highly not recommend this hotel very impolite stuff very noisy room very old furniture terrible smell in room hope hotel saved a lot of money for my days accommodation to provide uncomfortable room all best nothing']

In [46]:
#vectorizing the tweet by the pre-fitted tokenizer instance
twt = tokenizer.texts_to_sequences(reviews)
#padding the tweet to have exactly the same shape as `embedding_2` input
twt = pad_sequences(twt, maxlen=200, dtype='int32', value=0)
print(twt)
sentiment = loaded_model.predict(twt,batch_size=1,verbose = 2)[0]
if(np.argmax(sentiment) == 0):
    print("negative")
elif (np.argmax(sentiment) == 1):
    print("positive")

[[   0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    8    3  437   22    6   22  125   21    1
   240   11  134    6   35  100  357   93   34    1  240    8  108  273
     4  265   32    6  175   79   17   29 1115  105   85  134    8  108
   645  103    4  265    1    6  175  630  136   13   38 1573   23    1
   227   88  101  221  327  714  712    7    1   14    8  166   24  818
    27   13    6   17   75   82  805  499    9  453    8  114  645   77
   175   70   24  265   32    6    8  104  231   91  919   12   14    2
   175  114   66    8  100  158   21    8   79   17   22  742   10  224
    30   14   11 1755  453   11  134    6   11   96  336  244  248    7
     6 1408   14    5  247    9  107   12   32  230  813    4  6